In [ ]:
#for Google Colab
import sys
from google.colab import drive
from IPython.display import Javascript
from pprint import pprint

drive.mount('/content/drive')

###---- ROOT_PATHを記載 ----###
ROOT_PATH = '/content/drive/MyDrive/Scraping/'
sys.path.append(ROOT_PATH)

Mounted at /content/drive


In [ ]:
!pip install -q trafilatura
!pip install -q beautifulsoup4
!pip install -q requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.2 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup

# スクレイピングするURL
url = 'https://superceo.jp/list/'

# requestsを使ってURLからHTMLを取得
response = requests.get(url)

# ステータスコードが200であれば、正常にページが取得できている
if response.status_code == 200:
    # BeautifulSoupを使ってHTMLを解析
    soup = BeautifulSoup(response.text, 'html.parser')

    # HTMLの中身を整形して表示
    print(soup.prettify())
else:
    print("Failed to retrieve content")


<!DOCTYPE html>
<html>
 <head>
  <!-- tag -->
  <meta content="ja" name="language"/>
  <meta charset="utf-8"/>
  <!-- Google Tag Manager -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-K2P7FL');
  </script>
  <!-- End Google Tag Manager -->
  <script charset="utf-8" src="//typesquare.com/3/tsst/script/ja/typesquare.js?c2~~ZzkkVH0%3D&amp;fadein=1" type="text/javascript">
  </script>
  <link href="//netdna.bootstrapcdn.com/font-awesome/4.0.3/css/font-awesome.min.css" rel="stylesheet"/>
  <script src="//ajax.googleapis.com/ajax/libs/jquery/1.11.3/jquery.min.js">
  </script>
  <script src="/js/modernizr-svg.js" type="text/javascript">
  </script>
  <script src="/js/jquery.easing.1.3.js" type="t

In [ ]:
# すべての<a>タグを見つける
# for link in soup.find_all('a'):
#     print(link.get('href'))

base_url = 'https://superceo.jp'
urls = []
# '/company'で始まるすべての<a>タグを見つける
for link in soup.find_all('a', href=True):
    href = link.get('href')
    if href.startswith('/company'):
        full_url = base_url + href
        urls.append(full_url)
        print(full_url)  # 確認用にURLを表示


https://superceo.jp/company/z31002-sbc-medical-group
https://superceo.jp/company/b11007-ashitanoteam
https://superceo.jp/company/z210802-touchtogo
https://superceo.jp/company/i60202-life-design-partners
https://superceo.jp/company/i60905-nagaomi
https://superceo.jp/company/a01031-kizunasincerite
https://superceo.jp/company/g10006-abe-photography
https://superceo.jp/company/i210602-zeq
https://superceo.jp/company/g10006-abe-photography
https://superceo.jp/company/g10006-abe-photography
https://superceo.jp/company/i80702-azuma
https://superceo.jp/company/z60602-spree
https://superceo.jp/company/z90601-campfire
https://superceo.jp/company/i210801-ifaikeda
https://superceo.jp/company/z31204-golf-digest-online
https://superceo.jp/company/z30806-hoppy-beverage
https://superceo.jp/company/z30806-hoppy-beverage
https://superceo.jp/company/z31203-euglena
https://superceo.jp/company/i60706-isobehyakkaentokugawaten
https://superceo.jp/company/z70601-yohobrewing
https://superceo.jp/company/i61107-

In [ ]:
urls = list(set(urls))
len(urls)

247

In [ ]:
import requests
import csv, json, os, time
from bs4 import BeautifulSoup
from trafilatura import fetch_url, extract


output_dir = ROOT_PATH + "jsonl/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


def fetch_all_pages_to_jsonl(jsonl_filename):
    filename = os.path.join(output_dir, jsonl_filename)
    # 抽出したページ数のカウント
    count = 0
    # 全ページ数のカウント
    total_count = 0

    with open(filename, 'w', encoding='utf_8_sig') as file:

      for url in urls:
        total_count += 1
        page_url = url
        downloaded = fetch_url(page_url)
        soup = BeautifulSoup(downloaded, 'html.parser')
        title = soup.title.text if soup.title else ''
        extracted_text = extract(downloaded, include_tables=False, target_lang='ja')

        # "。"を含まないページはフィルタリング（単語のみや英語のページが多いため）
        if not extracted_text or "。" not in extracted_text:
            # print(extracted_text)
            continue

        # jsonl形式で書き込み
        page_data = {'title': title, 'url': page_url, 'text': extracted_text}
        json.dump(page_data, file, ensure_ascii=False)
        file.write('\n')

        count += 1
        print(f"Processed {count} pages.") if count % 1000 == 0 else None
        time.sleep(1)

    # 抽出したページ数を表示
    print(f"Total pages retrieved: {count}/ {total_count}")

fetch_all_pages_to_jsonl('superceo.jsonl')


Total pages retrieved: 247/ 247
